In [13]:
!pip install openpyxl
!pip install matplotlib
!pip install seaborn

import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.models as models
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [14]:
# Import data
brain_slices = np.load('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/revised_stitched_ws_images_v5.npy',allow_pickle=True)
df = pd.read_excel('/rds/projects/b/bowmanh-msc-project-group-2023to2024/Yifei_/Patients_CathyExport_processed_v5.xlsx', sheet_name='master_run')

In [15]:
patient_data_nn = df[:1072]
scanners_to_exclude = ['SON', 'NCL', 'EXT', 'UCH']
training_list = [1,2,3,4]
test_list = [5]
s_train_val = brain_slices[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]
s_test = brain_slices[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]

print("X_train_val:", s_train_val.shape)
print("X_test:", s_test.shape)

labels_train = patient_data_nn[patient_data_nn.NewGroups.isin(training_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']
labels_test = patient_data_nn[patient_data_nn.NewGroups.isin(test_list) & ~patient_data_nn.Scanner.isin(scanners_to_exclude)]['SpkPicDescTOTTS']

y_train_val = (labels_train>=60).astype(int)
y_test = (labels_test>=60).astype(int)

print("y_train_val:", len(y_train_val))
print("y_test:", len(y_test))

X_train_val: (608, 632, 760)
X_test: (150, 632, 760)
y_train_val: 608
y_test: 150


In [16]:
# Preprocess the image data
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_slices(slices):
    tensors = [preprocess(torch.tensor(slice, dtype=torch.float).unsqueeze(0)) for slice in slices]
    return torch.stack(tensors)

X_train_test = preprocess_slices(s_train_val).to(device)
y_train_test = torch.from_numpy(y_train_val.values).long().to(device)
X_test = preprocess_slices(s_test).to(device)
y_test = torch.from_numpy(y_test.values).long().to(device)

print(f"Number of Training/Validation: {len(y_train_test)}")
print(f"Number of Testing: {len(y_test)}")

# Preprocess the image data (assuming preprocess_slices function is defined)
print(f"Train/Validation Images Shape: {X_train_test.shape}")
print(f"Test Images Shape: {X_test.shape}")

Number of Training/Validation: 608
Number of Testing: 150
Train/Validation Images Shape: torch.Size([608, 3, 256, 307])
Test Images Shape: torch.Size([150, 3, 256, 307])


In [17]:
# Define the model
class_names = ['below','above']

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.3),
    torch.nn.Linear(256, len(class_names))
)

model = model.to(device)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)

/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/rds/homes/y/yxm326/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True


class EarlyStoppingAvgLoss:
    def __init__(self, patience=5, avg_len=10):
        self.patience = patience
        self.avg_len = avg_len
        self.best_avg_loss = float('inf')
        self.losses = []
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        self.losses.append(val_loss)
        if len(self.losses) > self.avg_len:
            self.losses = self.losses[-self.avg_len:]
        avg_loss = sum(self.losses) / len(self.losses)
        if avg_loss < self.best_avg_loss:
            self.best_avg_loss = avg_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True


class EarlyStoppingWeight:
    def __init__(self, patience=5, alpha=0.5):
        self.patience = patience
        self.alpha = alpha
        self.best_avg_loss = float('inf')
        self.avg_loss = 0
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.avg_loss == 0:
            self.avg_loss = val_loss
        else:
            self.last_loss = self.avg_loss
            self.avg_loss = self.alpha * val_loss + (1 - self.alpha) * self.avg_loss
        if self.avg_loss < self.best_avg_loss:
            self.best_avg_loss = self.avg_loss
        elif val_loss < self.last_loss:
            self.counter = 0
        else:
            self.counter += 1
#             print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
#                 print('INFO: Early stopping')
                self.early_stop = True

In [19]:
class EarlyStoppingGL:
    def __init__(self, alpha=0.5):
        """
        Initialize the early stopping criteria based on Generalization Loss (GL).

        Parameters:
        alpha (float): Threshold for the GL to trigger early stopping.
        """
        self.alpha = alpha
        self.best_loss = float('inf')
        self.early_stop = False
        self.stop_epoch = None
        self.val_losses = []

    def GL(self, val_loss, min_val_loss):
        return 100 * (val_loss / min_val_loss - 1)

    def __call__(self, val_loss):
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        GL_value = self.GL(val_loss, self.best_loss)
        if GL_value > self.alpha:
            self.early_stop = True
            self.stop_epoch = len(self.val_losses)

In [20]:
class EarlyStoppingGLPQRatio:
    def __init__(self, alpha=0.95, k=5):
        """
        Initialize the early stopping criteria based on GL/PQ ratio.

        Parameters:
        alpha (float): Threshold for the GL/PQ ratio to trigger early stopping.
        k (int): Number of epochs over which to calculate the PQ value.
        """
        self.alpha = alpha
        self.k = k
        self.best_loss = float('inf')
        self.val_losses = []
        self.train_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def GL(self, val_loss, min_val_loss):
        return 1000 * (val_loss / min_val_loss - 1)

    def PQ(self, train_losses):
        min_train_loss = min(train_losses)
        avg_train_loss = sum(train_losses) / len(train_losses)
        return 100 * (avg_train_loss / min_train_loss - 1)

    def __call__(self, val_loss, train_loss):
        self.val_losses.append(val_loss)
        self.train_losses.append(train_loss)

        if val_loss < self.best_loss:
            self.best_loss = val_loss

        else:
            GL_value = self.GL(val_loss, self.best_loss)
            PQ_value = self.PQ(self.train_losses[-self.k:])
            if GL_value / PQ_value > self.alpha:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [21]:
class EarlyStoppingValLoss:
    def __init__(self, s=20):
        """
        Initialize the early stopping criteria based on consecutive increases in validation loss.

        Parameters:
        s (int): Number of consecutive epochs with increasing validation loss to trigger early stopping.
        """
        self.s = s
        self.best_loss = float('inf')
        self.val_losses = []
        self.early_stop = False
        self.stop_epoch = None

    def __call__(self, val_loss):
        """
        Check if training should be stopped based on consecutive increases in validation loss.

        Parameters:
        val_loss (float): Current validation loss.
        """
        self.val_losses.append(val_loss)
        if val_loss < self.best_loss:
            self.best_loss = val_loss

        if len(self.val_losses) > self.s:
            increasing = all(x < y for x, y in zip(self.val_losses[-self.s:], self.val_losses[-self.s + 1:]))
            if increasing:
                self.early_stop = True
                self.stop_epoch = len(self.val_losses)

In [22]:
class EarlyStoppingCross:
    def __init__(self):
        """
        Initialize the early stopping criteria based on the last crossing point
        of the train loss and validation loss curves after training for 100 epochs.
        """
        self.train_losses = []
        self.val_losses = []
        self.best_epoch = None

    def __call__(self, train_loss, val_loss):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)

        if len(self.train_losses) >= 100:
            for epoch in range(99, -1, -1):
                if self.train_losses[epoch] > self.val_losses[epoch]:
                    self.best_epoch = epoch + 1
                    break

    def get_best_epoch(self):
        return self.best_epoch

In [23]:
best_val_loss = float('inf')   # Initialize the best validation loss to positive infinity
model_path = 'best_model.pth'  # Path to save the best model weights
n_epochs = 50
# early_stopping = EarlyStopping(patience=20, min_delta=0.01)
# early_stopping = EarlyStoppingAvgLoss(patience=20, avg_len=3)
# early_stopping = EarlyStoppingEWMA(patience=10, alpha=0.5)

# early_stopping_gl = EarlyStoppingGL()
# early_stopping_glqr = EarlyStoppingGLPQRatio()
# early_stopping_val_loss = EarlyStoppingValLoss()

In [11]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold, train_test_split

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output.data, 1)
        total_train += target.size(0)
        correct_train += (predicted == target).sum().item()
    
    return train_loss / len(train_loader.dataset), 100 * correct_train / total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output.data, 1)
            total_val += target.size(0)
            correct_val += (predicted == target).sum().item()
    
    return val_loss / len(val_loader.dataset), 100 * correct_val / total_val

def train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs):
    model = create_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
    early_stopping = EarlyStoppingWeight(patience=patience, alpha=alpha)
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(n_epochs):
        train_loss, train_accuracy = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_accuracy = validate(model, val_loader, criterion)
        
        print(f'  Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"    Early stopping at epoch {epoch + 1}")
            break
    
    return best_model_state, best_val_loss, val_accuracy

def test_model(model, X_test, y_test):
    model.eval()
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.CrossEntropyLoss()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            test_loss += loss.item() * data.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100 * correct / total
    return test_loss, accuracy

def run_experiment(X, y, X_test, y_test, alpha, patience, num_runs, n_epochs, k_folds):
    all_run_results = []

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=run)
        
        run_fold_losses = []
        run_fold_accuracies = []
        best_run_model = None
        best_run_loss = float('inf')

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f'Fold {fold + 1}')
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            best_model_state, best_val_loss, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs)
            run_fold_losses.append(best_val_loss)
            run_fold_accuracies.append(val_accuracy)
            print(f'Fold {fold + 1}: Best Validation Loss: {best_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

            if best_val_loss < best_run_loss:
                best_run_loss = best_val_loss
                best_run_model = best_model_state

        print(f"====== Run {run + 1} Information ====== ")
        avg_fold_loss = sum(run_fold_losses) / k_folds
        avg_fold_accuracy = sum(run_fold_accuracies) / k_folds
        print(f'Run {run + 1} Average Validation Loss: {avg_fold_loss:.4f}, Average Validation Accuracy: {avg_fold_accuracy:.2f}%')

        # Test the best model for this run
        best_model = create_model()
        best_model.load_state_dict(best_run_model)
        test_loss, test_accuracy = test_model(best_model, X_test, y_test)
        print(f"Run {run + 1} Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

        print(f"====== Run {run + 1} End ====== ")

        
        all_run_results.append({
            'run': run + 1,
            'fold_losses': run_fold_losses,
            'fold_accuracies': run_fold_accuracies,
            'avg_val_loss': avg_fold_loss,
            'avg_val_accuracy': avg_fold_accuracy,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy
        })

    return all_run_results

# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 50
    alpha = 0.4
    patience = 9
    n_epochs = 50
    k_folds = 4

    run_results = run_experiment(
        X_train_test, y_train_test, X_test, y_test, alpha, patience, num_runs, n_epochs, k_folds
    )

    with open('UP_result_detailed.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Fold', 'Validation Loss', 'Validation Accuracy', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            for fold in range(k_folds):
                writer.writerow([
                    result['run'],
                    fold + 1,
                    result['fold_losses'][fold],
                    result['fold_accuracies'][fold],
                    result['avg_val_loss'],
                    result['avg_val_accuracy'],
                    result['test_loss'],
                    result['test_accuracy']
                ])

    with open('UP_result_summary.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            writer.writerow([
                result['run'],
                result['avg_val_loss'],
                result['avg_val_accuracy'],
                result['test_loss'],
                result['test_accuracy']
            ])

    print("Detailed results saved to 'UP_result_detailed.csv'")
    print("Summary results saved to 'UP_result_summary.csv'")

Run 1/50
Fold 1
  Epoch 1: Train Loss: 0.6456, Val Loss: 0.6770, Train Accuracy: 63.60%, Val Accuracy: 64.47%
  Epoch 2: Train Loss: 0.6259, Val Loss: 0.6663, Train Accuracy: 67.11%, Val Accuracy: 64.47%
  Epoch 3: Train Loss: 0.6014, Val Loss: 0.6464, Train Accuracy: 67.98%, Val Accuracy: 64.47%
  Epoch 4: Train Loss: 0.5994, Val Loss: 0.6408, Train Accuracy: 67.11%, Val Accuracy: 64.47%
  Epoch 5: Train Loss: 0.5772, Val Loss: 0.6358, Train Accuracy: 68.20%, Val Accuracy: 64.47%
  Epoch 6: Train Loss: 0.5655, Val Loss: 0.6352, Train Accuracy: 68.86%, Val Accuracy: 64.47%
  Epoch 7: Train Loss: 0.5460, Val Loss: 0.6288, Train Accuracy: 67.11%, Val Accuracy: 65.13%
  Epoch 8: Train Loss: 0.5167, Val Loss: 0.6243, Train Accuracy: 72.81%, Val Accuracy: 67.76%
  Epoch 9: Train Loss: 0.5079, Val Loss: 0.6211, Train Accuracy: 74.12%, Val Accuracy: 67.76%
  Epoch 10: Train Loss: 0.4901, Val Loss: 0.6228, Train Accuracy: 73.46%, Val Accuracy: 67.76%
  Epoch 11: Train Loss: 0.4684, Val Loss: 0

KeyboardInterrupt: 

In [12]:
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        predicted = (output > 0).float()
        total_train += target.size(0) * target.size(1)
        correct_train += (predicted == target).float().sum()
    
    return train_loss / len(train_loader.dataset), 100 * correct_train / total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
            
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            predicted = (output > 0).float()
            total_val += target.size(0) * target.size(1)
            correct_val += (predicted == target).float().sum()
    
    return val_loss / len(val_loader.dataset), 100 * correct_val / total_val

def train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs):
    model = create_model()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
    early_stopping = EarlyStoppingGLPQRatio(alpha=alpha, k=patience)
    
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32)
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(n_epochs):
        train_loss, train_accuracy = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_accuracy = validate(model, val_loader, criterion)
        
        print(f'  Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
        
        early_stopping(val_loss, train_loss)
        if early_stopping.early_stop:
            print(f"    Early stopping at epoch {epoch + 1}")
            break
    
    return best_model_state, best_val_loss, val_accuracy

def test_model(model, X_test, y_test):
    model.eval()
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.BCEWithLogitsLoss()
    test_loss, test_accuracy = validate(model, test_loader, criterion)
    return test_loss, test_accuracy

def run_experiment(X, y, X_test, y_test, alpha, patience, num_runs, n_epochs, k_folds):
    all_run_results = []

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=run)
        
        run_fold_losses = []
        run_fold_accuracies = []
        best_run_model = None
        best_run_loss = float('inf')

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f'Fold {fold + 1}')
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            best_model_state, best_val_loss, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, alpha, patience, n_epochs)
            run_fold_losses.append(best_val_loss)
            run_fold_accuracies.append(val_accuracy)
            
            print(f'Fold {fold + 1}: Best Validation Loss: {best_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

            if best_val_loss < best_run_loss:
                best_run_loss = best_val_loss
                best_run_model = best_model_state

        print(f"====== Run {run + 1} Information ====== ")
        avg_fold_loss = sum(run_fold_losses) / k_folds
        avg_fold_accuracy = sum(run_fold_accuracies) / k_folds
        print(f'Run {run + 1} Average Validation Loss: {avg_fold_loss:.4f}, Average Validation Accuracy: {avg_fold_accuracy:.2f}%')

        # Test the best model for this run
        best_model = create_model()
        best_model.load_state_dict(best_run_model)
        test_loss, test_accuracy = test_model(best_model, X_test, y_test)
        print(f"Run {run + 1} Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        print(f"====== Run {run + 1} End ====== ")

        all_run_results.append({
            'run': run + 1,
            'fold_losses': run_fold_losses,
            'fold_accuracies': run_fold_accuracies,
            'avg_val_loss': avg_fold_loss,
            'avg_val_accuracy': avg_fold_accuracy,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy
        })

    return all_run_results

# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 50
    alpha = 1.3
    patience = 10
    n_epochs = 50
    k_folds = 4

    run_results = run_experiment(X_train_test, y_train_test, X_test, y_test, alpha, patience, num_runs, n_epochs, k_folds)

    with open('PQ_result_detailed.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Fold', 'Validation Loss', 'Validation Accuracy', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            for fold in range(k_folds):
                writer.writerow([
                    result['run'],
                    fold + 1,
                    result['fold_losses'][fold],
                    result['fold_accuracies'][fold],
                    result['avg_val_loss'],
                    result['avg_val_accuracy'],
                    result['test_loss'],
                    result['test_accuracy']
                ])

    with open('PQ_result_summary.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            writer.writerow([
                result['run'],
                result['avg_val_loss'],
                result['avg_val_accuracy'],
                result['test_loss'],
                result['test_accuracy']
            ])

    print("Detailed results saved to 'PQ_result_detailed.csv'")
    print("Summary results saved to 'PQ_result_summary.csv'")

Run 1/50
Fold 1
  Epoch 1: Train Loss: 0.6619, Val Loss: 0.6606, Train Accuracy: 62.50%, Val Accuracy: 64.47%
  Epoch 2: Train Loss: 0.6336, Val Loss: 0.6579, Train Accuracy: 66.45%, Val Accuracy: 64.47%
  Epoch 3: Train Loss: 0.6205, Val Loss: 0.6616, Train Accuracy: 67.00%, Val Accuracy: 64.47%
    Early stopping at epoch 3
Fold 1: Best Validation Loss: 0.6579, Accuracy: 64.47%
Fold 2
  Epoch 1: Train Loss: 0.7080, Val Loss: 0.6726, Train Accuracy: 49.45%, Val Accuracy: 57.89%
  Epoch 2: Train Loss: 0.6608, Val Loss: 0.6382, Train Accuracy: 62.17%, Val Accuracy: 71.71%
  Epoch 3: Train Loss: 0.6427, Val Loss: 0.6116, Train Accuracy: 64.04%, Val Accuracy: 72.37%
  Epoch 4: Train Loss: 0.6388, Val Loss: 0.5867, Train Accuracy: 64.25%, Val Accuracy: 72.37%
  Epoch 5: Train Loss: 0.6275, Val Loss: 0.5878, Train Accuracy: 64.58%, Val Accuracy: 72.37%
  Epoch 6: Train Loss: 0.6258, Val Loss: 0.5856, Train Accuracy: 64.36%, Val Accuracy: 72.37%
  Epoch 7: Train Loss: 0.6212, Val Loss: 0.582

In [53]:
import csv
import collections
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.model_selection import KFold

# Assume necessary imports and definitions (EarlyStoppingWeight, device, class_names, etc.)

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        predicted = (output > 0).float()
        total_train += target.size(0) * target.size(1)
        correct_train += (predicted == target).float().sum()
    
    return train_loss / len(train_loader.dataset), 100 * correct_train / total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
            
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            predicted = (output > 0).float()
            total_val += target.size(0) * target.size(1)
            correct_val += (predicted == target).float().sum()
    
    return val_loss / len(val_loader.dataset), 100 * correct_val / total_val


def train_and_validate(X_train, y_train, X_val, y_val, alpha, n_epochs):
    model = create_model()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0005, momentum=0.9)
    early_stopping = EarlyStoppingGL(alpha=alpha)
    
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=32)
    
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(n_epochs):
        train_loss, train_accuracy = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_accuracy = validate(model, val_loader, criterion)
        
        print(f'  Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.2f}%, Val Accuracy: {val_accuracy:.2f}%')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print(f"    Early stopping at epoch {epoch + 1}")
            break
    
    return best_model_state, best_val_loss, val_accuracy

def test_model(model, X_test, y_test):
    model.eval()
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.BCEWithLogitsLoss()
    test_loss, test_accuracy = validate(model, test_loader, criterion)
    return test_loss, test_accuracy

def run_experiment(X, y, X_test, y_test, alpha, num_runs, n_epochs, k_folds):
    all_run_results = []

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=run)
        
        run_fold_losses = []
        run_fold_accuracies = []
        best_run_model = None
        best_run_loss = float('inf')

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f'Fold {fold + 1}')
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            best_model_state, best_val_loss, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, alpha, n_epochs)
            run_fold_losses.append(best_val_loss)
            run_fold_accuracies.append(val_accuracy)
            
            print(f'Fold {fold + 1}: Best Validation Loss: {best_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

            if best_val_loss < best_run_loss:
                best_run_loss = best_val_loss
                best_run_model = best_model_state

        print(f"====== Run {run + 1} Information ====== ")
        avg_fold_loss = sum(run_fold_losses) / k_folds
        avg_fold_accuracy = sum(run_fold_accuracies) / k_folds
        print(f'Run {run + 1} Average Validation Loss: {avg_fold_loss:.4f}, Average Validation Accuracy: {avg_fold_accuracy:.2f}%')

        # Test the best model for this run
        best_model = create_model()
        best_model.load_state_dict(best_run_model)
        test_loss, test_accuracy = test_model(best_model, X_test, y_test)
        print(f"Run {run + 1} Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        print(f"====== Run {run + 1} End ====== ")

        all_run_results.append({
            'run': run + 1,
            'fold_losses': run_fold_losses,
            'fold_accuracies': run_fold_accuracies,
            'avg_val_loss': avg_fold_loss,
            'avg_val_accuracy': avg_fold_accuracy,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy
        })

    return all_run_results

        
# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 50
    alpha = 1.1
    k_folds = 4
    n_epochs = 50

    run_results = run_experiment(X_train_test, y_train_test, X_test, y_test, alpha, num_runs, n_epochs, k_folds)

    with open('GL_result_detailed.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Fold', 'Validation Loss', 'Validation Accuracy', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            for fold in range(k_folds):
                writer.writerow([
                    result['run'],
                    fold + 1,
                    result['fold_losses'][fold],
                    result['fold_accuracies'][fold],
                    result['avg_val_loss'],
                    result['avg_val_accuracy'],
                    result['test_loss'],
                    result['test_accuracy']
                ])

    with open('GL_result_summary.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Run', 'Avg Validation Loss', 'Avg Validation Accuracy', 'Test Loss', 'Test Accuracy'])
        for result in run_results:
            writer.writerow([
                result['run'],
                result['avg_val_loss'],
                result['avg_val_accuracy'],
                result['test_loss'],
                result['test_accuracy']
            ])

    print("Detailed results saved to 'GL_result_detailed.csv'")
    print("Summary results saved to 'GL_result_summary.csv'")

Run 1/50
Fold 1
  Epoch 1: Train Loss: 0.6799, Val Loss: 0.6650, Train Accuracy: 57.13%, Val Accuracy: 64.47%
  Epoch 2: Train Loss: 0.6392, Val Loss: 0.6565, Train Accuracy: 65.90%, Val Accuracy: 64.47%
  Epoch 3: Train Loss: 0.6280, Val Loss: 0.6604, Train Accuracy: 67.00%, Val Accuracy: 64.47%
  Epoch 4: Train Loss: 0.6161, Val Loss: 0.6513, Train Accuracy: 67.11%, Val Accuracy: 64.47%
  Epoch 5: Train Loss: 0.6125, Val Loss: 0.6485, Train Accuracy: 67.11%, Val Accuracy: 64.47%
  Epoch 6: Train Loss: 0.6043, Val Loss: 0.6469, Train Accuracy: 67.00%, Val Accuracy: 64.47%
  Epoch 7: Train Loss: 0.6020, Val Loss: 0.6419, Train Accuracy: 67.11%, Val Accuracy: 64.47%
  Epoch 8: Train Loss: 0.5880, Val Loss: 0.6404, Train Accuracy: 67.32%, Val Accuracy: 64.47%
  Epoch 9: Train Loss: 0.5905, Val Loss: 0.6422, Train Accuracy: 67.21%, Val Accuracy: 64.47%
  Epoch 10: Train Loss: 0.5840, Val Loss: 0.6359, Train Accuracy: 67.21%, Val Accuracy: 64.47%
  Epoch 11: Train Loss: 0.5707, Val Loss: 0

## CROSS

In [25]:
import csv
import collections
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.model_selection import KFold

# Assume necessary imports and definitions (device, class_names, etc.)

class EarlyStoppingCross:
    def __init__(self):
        """
        Initialize the early stopping criteria based on the last crossing point
        of the train loss and validation loss curves after training for 100 epochs.
        """
        self.train_losses = []
        self.val_losses = []
        self.best_epoch = None

    def __call__(self, train_loss, val_loss):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)

        if len(self.train_losses) >= 100:
            for epoch in range(99, -1, -1):
                if self.train_losses[epoch] > self.val_losses[epoch]:
                    self.best_epoch = epoch + 1
                    break

    def get_best_epoch(self):
        return self.best_epoch

def create_model():
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, len(class_names))
    )
    return model.to(device)

def train_epoch(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * data.size(0)
        predicted = (output > 0).float()
        total_train += target.size(0) * target.size(1)
        correct_train += (predicted == target).float().sum()
    
    return train_loss, correct_train, total_train

def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            target = nn.functional.one_hot(target, num_classes=len(class_names)).float()
            
            output = model(data)
            loss = criterion(output, target)
            
            val_loss += loss.item() * data.size(0)
            predicted = (output > 0).float()
            total_val += target.size(0) * target.size(1)
            correct_val += (predicted == target).float().sum()
    
    return val_loss, correct_val, total_val

def train_and_validate(X_train, y_train, X_val, y_val, n_epochs):
    model = create_model()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
    # early_stopping = EarlyStoppingCross()

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_val, y_val), batch_size=32)

    train_losses, val_losses = [], []
    val_accuracies = []
    best_epoch = None

    for epoch in range(n_epochs):
        train_loss, correct_train, total_train = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, correct_val, total_val = validate(model, val_loader, criterion)

        val_accuracy = 100 * correct_val / total_val
        val_accuracies.append(val_accuracy)
        
        train_losses.append(train_loss / len(train_loader.dataset))
        val_losses.append(val_loss / len(val_loader.dataset))

    if len(train_losses) >= 100:
        for epoch in range(99, -1, -1):
            if train_losses[epoch] > val_losses[epoch]:
                best_epoch = epoch + 1
                print("===Early Stopped===")
                break

    if best_epoch == None:
        best_epoch = val_losses.index(min(val_losses)) + 1
        
    print(f"    Best epoch: {best_epoch}, Val Accuracy: {val_accuracies[best_epoch]:.2f}%")

    best_model = create_model()
    best_model.load_state_dict(model.state_dict())

    return best_model, val_losses[best_epoch], val_accuracies[best_epoch]

def test_model(model, X_test, y_test):
    test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=32)
    criterion = nn.BCEWithLogitsLoss()
    _, correct_test, total_test = validate(model, test_loader, criterion)
    test_accuracy = 100 * correct_test / total_test
    return test_accuracy

def run_experiment(X_train_test, y_train_test, num_runs, outer_k, inner_k, n_epochs):
    all_val_accuracies = [[] for _ in range(num_runs)]

    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs}")
        best_val_loss = float('inf')
        outer_kfold = KFold(n_splits=outer_k, shuffle=False)

        for outer_fold, (train_val_idx, test_idx) in enumerate(outer_kfold.split(X_train_test)):
            print(f'Outer Fold {outer_fold + 1}')
            X_train_val, y_train_val = X_train_test[train_val_idx], y_train_test[train_val_idx]
            X_test, y_test = X_train_test[test_idx], y_train_test[test_idx]

            inner_kfold = KFold(n_splits=inner_k, shuffle=False)
            train_idx, val_idx = next(inner_kfold.split(X_train_val))
            X_train, y_train = X_train_val[train_idx], y_train_val[train_idx]
            X_val, y_val = X_train_val[val_idx], y_train_val[val_idx]

            model, val_loss, val_accuracy = train_and_validate(X_train, y_train, X_val, y_val, n_epochs)
            all_val_accuracies[run].append(val_accuracy)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_val_accuracy = val_accuracy
                
        print("=============== Info ===================")
        print(f'Best Val Loss: {best_val_loss:.4f}')
        print(f'Val Accuracy in Best Val Loss: {best_val_accuracy:.2f}%')
        print("=============== End ===================")

    return all_val_accuracies

def write_results_to_csv(all_val_accuracies, num_runs, outer_k):
    with open('experiment_results.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        # Write header
        header = ['Run', 'Outer Fold', 'Test Accuracy']
        csvwriter.writerow(header)
        
        # Write data
        for run in range(num_runs):
            for outer_fold in range(outer_k):
                accuracy = all_val_accuracies[run][outer_fold]
                row = [run + 1, outer_fold + 1, f"{accuracy:.2f}"]
                csvwriter.writerow(row)
        
        print("Results have been written to experiment_results.csv")
        
# Main execution
if __name__ == "__main__":
    # Define hyperparameters
    num_runs = 20
    outer_k = 4
    inner_k = 1
    n_epochs = 100

    # Assume X_train_test and y_train_test are loaded here

    all_val_accuracies = run_experiment(
        X_train_test, y_train_test, num_runs, outer_k, inner_k, n_epochs
    )

    # Write results to CSV
    write_results_to_csv(all_val_accuracies, num_runs, outer_k)

Run 1/20
Outer Fold 1


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.